In [20]:
import psycopg2 as postgres

BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()

In [7]:
# Question 1
cur.execute(open("sql/questions/question_1#1.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Les 10 villes les plus peuplées de France en 2018:')
for ville in data:
    print(f'\t- {ville[0]}: {ville[1]:,} habitants')


In [28]:
# Question 2
cur.execute(open("sql/questions/question_1#2.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Évolution de la population Française au fil des années:')
for population in data:
    population = [0 if v is None else v for v in population]
    print(f'\t- Année {population[0]}: {population[1]:,} habitants -> +{population[2]:,} habitants ({population[3]}%)')


Évolution de la population Française au fil des années:
	- Année 1968: 50,798,112 habitants -> +0 habitants (0%)
	- Année 1975: 53,764,064 habitants -> +2,965,952 habitants (5.516606780320774%)
	- Année 1982: 55,569,542 habitants -> +1,805,478 habitants (3.2490424340729676%)
	- Année 1990: 58,040,659 habitants -> +2,471,117 habitants (4.257561927406786%)
	- Année 1999: 60,149,901 habitants -> +2,109,242 habitants (3.506642513010952%)
	- Année 2008: 63,961,859 habitants -> +3,811,958 habitants (5.959736098351988%)
	- Année 2013: 65,564,756 habitants -> +1,602,897 habitants (2.444754007778203%)
	- Année 2018: 66,732,538 habitants -> +1,167,782 habitants (1.7499439328982211%)


In [29]:
# Question 3
cur.execute(open("sql/questions/question_1#3.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Ville la plus peuplée de chaque région en 2018:')
for population in data:
    print(f'\t- Région {population[2]} -> {population[1]} ({population[0]}): {population[3]:,} habitants')


[('75056', 'Paris', 'ILE DE FRANCE', 2175601),
 ('13055', 'Marseille', 'PROVENCE ALPES COTE D AZUR', 868277),
 ('69123', 'Lyon', 'AUVERGNE RHONE ALPES', 518635),
 ('31555', 'Toulouse', 'OCCITANIE', 486828),
 ('44109', 'Nantes', 'PAYS DE LA LOIRE', 314138),
 ('67482', 'Strasbourg', 'GRAND EST', 284677),
 ('33063', 'Bordeaux', 'NOUVELLE AQUITAINE', 257068),
 ('59350', 'Lille', 'HAUTS DE FRANCE', 233098),
 ('35238', 'Rennes', 'BRETAGNE', 217728),
 ('76351', 'Le Havre', 'NORMANDIE', 169733),
 ('21231', 'Dijon', 'BOURGOGNE FRANCHE COMTE', 156854),
 ('97411', 'Saint-Denis', 'LA REUNION', 150535),
 ('37261', 'Tours', 'CENTRE VAL DE LOIRE', 136463),
 ('97209', 'Fort-de-France', 'MARTINIQUE', 78126),
 ('2A004', 'Ajaccio', 'CORSE', 70817),
 ('97302', 'Cayenne', 'GUYANE', 63652),
 ('97101', 'Les Abymes', 'GUADELOUPE', 53082)]

In [9]:
conn.close()